In [ ]:
# CPU required for this notebooke: used cpu with High ram

In [ ]:
!pip install -q sentence-transformers faiss-cpu transformers datasets detoxify

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 104.8 MB/s eta 0:00:00


In [ ]:
import os
import pickle
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
rag_dir = '/content/drive/My Drive/kilt_rag_data'

Mounted at /content/drive


In [ ]:
paras_dir = os.path.join(rag_dir, "paras_v2")
os.makedirs(paras_dir, exist_ok=True)

faiss_dir = os.path.join(rag_dir, "faiss_index")
os.makedirs(faiss_dir, exist_ok=True)

In [ ]:
# === CONFIG ===
output_path = f"{faiss_dir}/index_to_paragraph_512.pkl"  # Output mapping file

# === INITIALIZE ===
index_to_paragraph = {}
global_index = 0

# === List all .pkl files sorted by chunk index ===
# chunk_files = sorted([
#     f for f in os.listdir(paras_dir)
#     if f.endswith(".pkl")
# ], key=lambda x: int(x.split("_")[-1].split(".")[0]))  # sort by chunk id

# print(f"Found {len(chunk_files)} .pkl files in {paras_dir}")

# === Loop and build mapping ===
for chunk_id in tqdm(range(60), desc="Streaming "):
    file_path = os.path.join(paras_dir, f"kilt_paras_chunk_{chunk_id}.pkl")

# for chunk_file in tqdm(chunk_files, desc="Mapping chunks"):
#     file_path = os.path.join(paras_dir, chunk_file)
    try:
        with open(file_path, "rb") as f:
            paragraphs = pickle.load(f)

        if not isinstance(paragraphs, list):
            print(f"Skipped {file_path} — not a list")
            continue

        for para in paragraphs:
            if isinstance(para, str) and para.strip():
                index_to_paragraph[global_index] = para.strip()
                global_index += 1

    except Exception as e:
        print(f"Error in {file_path}: {e}")


with open(output_path, "wb") as f:
    pickle.dump(index_to_paragraph, f)

print(f"Saved {len(index_to_paragraph)} paragraph mappings to: {output_path}")

Streaming to Qdrant: 100%|██████████| 60/60 [03:53<00:00,  3.89s/it]


Saved 5099120 paragraph mappings to: /content/drive/My Drive/kilt_rag_data/faiss_index/index_to_paragraph_512.pkl
